In [1]:
import numpy as np
from scipy.optimize import minimize

def portfolio_volatility(weights, cov_matrix):
    """Calculate the portfolio volatility."""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

def portfolio_return(weights, returns):
    """Calculate the portfolio return (negative for minimization)."""
    return -np.dot(weights, returns)

def optimize_portfolio(returns, cov_matrix, target_return=None, weight_bounds=None, volatility_bound=None):
    """
    Optimize portfolio weights to maximize returns while bounding volatility.
    
    Args:
    - returns: An n-dimensional vector of asset expected returns.
    - cov_matrix: An n x n covariance matrix of asset returns.
    - target_return: The desired portfolio return (if None, this constraint is not enforced).
    - weight_bounds: A list of tuples specifying the lower and upper bounds for each asset's weight.
    - volatility_bound: A target for the maximum portfolio volatility (if None, this constraint is not enforced).
    
    Returns:
    - res.x: Optimized weights of the portfolio.
    """
    n_assets = len(returns)
    
    # Define initial guess for asset weights (equal weighting)
    init_guess = np.ones(n_assets) / n_assets
    
    # Define the bounds for each asset weight (use default bounds if not provided)
    if weight_bounds is None:
        weight_bounds = [(0, 1) for _ in range(n_assets)]  # Default to long-only portfolio bounds
    
    # Define the constraints
    constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]  # Sum of weights must be 1
    
    if volatility_bound is not None:
        # Add inequality constraint for portfolio volatility (must be less than or equal to volatility_bound)
        constraints.append({'type': 'ineq', 'fun': lambda weights: volatility_bound - portfolio_volatility(weights, cov_matrix)})
    
    # Objective function: minimize negative portfolio return (i.e., maximize returns)
    result = minimize(portfolio_return, init_guess, args=(returns,), method='SLSQP', bounds=weight_bounds, constraints=constraints)
    
    return result

# Example Usage
if __name__ == "__main__":
    # Example inputs
    returns = np.array([0.08, 0.04, 0.07])  # Example expected returns
    volatilities = np.array([0.15, 0.05, 0.12]) 
    corr_matrix = np.array([[1, -0.2, 0.4], 
                           [-0.2, 1, 0.1], 
                           [0.4, 0.1, 1]])  # Example correlation matrix
    cov_matrix = np.diag(volatilities) @ corr_matrix @ np.diag(volatilities)
    
    # Example bounds for each asset (Asset 1: [0, 1], Asset 2: [0.1, 0.6], Asset 3: [0.05, 0.4])
    weight_bounds = [(0.4, 1), (0, 0.5), (0, 0.2)]
    
    # Optimize portfolio for a target return of 6% and volatility bound of 10%
    result = optimize_portfolio(returns, cov_matrix, target_return=0.06, weight_bounds=weight_bounds, volatility_bound=0.1)
    
    print("Optimized weights:", result.x)
    print("Portfolio volatility:", portfolio_volatility(result.x, cov_matrix))
    print("Portfolio return:", -portfolio_return(result.x, returns))


Optimized weights: [0.59353026 0.20646974 0.2       ]
Portfolio volatility: 0.10000057437048875
Portfolio return: 0.06974121027112826
